## Acoustic Mirror Orientation

http://localhost:8888/?token=sloth

In [ ]:
from tqdm import trange

# import torch
import numpy as np
import matplotlib.pyplot as plt

import drjit as dr
import mitsuba as mi

from libs import utils

plt.style.use('ggplot')

mi.set_variant('cuda_ad_acoustic')
mi.set_log_level(mi.LogLevel.Warn)

sess_seed   = 0 #np.random.randint(0, 2**30)
sess_seed_g = 1 #np.random.randint(0, 2**30)
print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
bin_t = 5
max_t = 0.3

tf          = mi.ScalarTransform4f
mic_pos     = np.array([ 0.,  0.,  5.])
speaker_pos = np.array([20.,  0.,  5 ])
sail_pose   = np.array([10., 20.,  5.])
sail_size   = np.array([10.,  2.,  8.]) / 2.

sail_vertex_base = mi.Transform4f.scale(sail_size).rotate([1., 0., 0.], angle=90.) @ mi.Point3f(
    [-1., -1.,  1.,  1.],
    [-1.,  1., -1.,  1.],
    [ 0.,  0.,  0.,  0.]
)

scene_dict = {
    "type": "scene",
    "integrator": {
        "type": "prb_reparam_acoustic",
        "max_depth": 2,
        "max_time":  max_t,
        "skip_direct": True,
    },

    "sensor": {
        "type": "microphone",
        "cos_cutoff": 0.8,
        "to_world": tf.look_at(
            origin=mic_pos,
            target=sail_pose,
            up=[0, 1, 0]
        ),
        "film": {
            "type": "tape",
            "wav_bins":  1,
            "time_bins": bin_t,
            "filter": { "type": "box", },
            "count": True
        },
        "sampler": { "type": "stratified", "sample_count": 2**20, },
    },

    "speaker": {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/sphere.ply",
        "to_world": tf.translate(speaker_pos).scale(2.0),
        "emitter": { "type": "area", "radiance": { "type": "uniform", "value": 1e0 } },
    },

    "sail": {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
        "bsdf": {
            # "type": "roughconductor",
            # "alpha": 0.05,

            "type": "acousticbsdf",
            "scattering": { "type": "spectrum", "value": 0.00 },
            "absorption": { "type": "spectrum", "value": 0.30 },

            # 'type': 'blendbsdf',
            # 'weight': 0.1,
            # 'bsdf_0': { 'type': 'conductor' },
            # 'bsdf_1': { 'type': 'diffuse' },
        },
    },
}

### Optimization Setup

In [ ]:
key = "sail.vertex_positions"

opt = mi.ad.Adam(lr=0.005)
opt['a'] = mi.Float(0.0)

def apply_transform(params, opt):
    opt['a'] = dr.clamp(opt['a'], -2., 2.)
    transf = mi.Transform4f.translate(sail_pose).rotate(
        axis=[0., 0., 1.],
        angle=(opt['a'] * 30.)
    )
    params[key] = dr.ravel(transf @ sail_vertex_base)
    params.update()

### Load and Render Scene

In [ ]:
scene  = mi.load_dict(scene_dict)
params = mi.traverse(scene)
opt['a'] = mi.Float(0.32)
apply_transform(params, opt)

img = mi.render(scene, seed=sess_seed)
utils.plot_hist(img[:, :, 0], max_time=max_t, time_bins=bin_t)

### Cos-Cutoff Visualization

In [ ]:
prb    = scene.integrator()
sensor = scene.sensors()[0]
sampler, spp   = prb.prepare(sensor, seed=0, spp=2**12)
ray, w, p, det = prb.sample_rays(scene=scene, sensor=sensor, sampler=sampler, reparam=None)

dirs = ray.d
X = dirs.x.numpy()
Y = dirs.y.numpy()
Z = dirs.z.numpy()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 5))
ax1.scatter(X, Y, s=1.0)
ax1.set_xlim(-1.1, 1.1)
ax1.set_ylim(-1.1, 1.1)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax2.scatter(X, Z, s=1.0)
ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1)
ax2.set_xlabel("x")
ax2.set_ylabel("z")
fig.show()

### Loss Values

In [ ]:
angles = np.linspace(-0.5, 0.5, 51, endpoint=True)
data = np.zeros((angles.shape[0], 2))

for i in trange(angles.shape[0]):
    opt['a'] = mi.Float(angles[i])
    apply_transform(params, opt)
    img = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g+i, spp=2**22)

    l   = dr.mean(img[:, :, 0] / 2**22)
    dr.backward(l)

    data[i, 0] = l[0]
    data[i, 1] = dr.grad(opt['a'])[0]

In [ ]:
i, j = np.sort(np.argsort(data[:, 0])[-2:])
p = angles[np.argmin(data[i:j, 0])]
# p = angles[np.argmax(data[:, 0])]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(angles, data[:, 0])
ax1.vlines(p, data[:, 0].min(), data[:, 0].max(), linestyle='dotted', color="k")
ax1.set_xlabel("opt['a']")
ax1.set_ylabel("loss")

ax2.plot(angles, data[:, 1])
ax2.vlines(p, data[:, 1].min(), data[:, 1].max(), linestyle='dashed', linewidth=1.0, color="k")
ax2.hlines(0, angles[0], angles[-1], linestyle='dotted', color="k", linewidth=1.0)
ax2.set_xlabel("opt['a']")
ax2.set_ylabel("grad")
plt.show()

In [ ]:
assert False

### Main Optimization Loop

In [ ]:
opt['a'] = mi.Float(-0.3)
angles, losses, grads = [], [], []

In [ ]:
iters = 50

for i in trange(iters) if iters > 1 else range(iters):
    apply_transform(params, opt)
    img = mi.render(scene, params, seed=sess_seed+i, seed_grad=sess_seed_g+i)

    l = -1. * dr.sum(img)
    dr.backward(l, flags=dr.ADFlag.ClearNone if iters < 2 else dr.ADFlag.Default)

    if iters < 2:
        # display(dr.unravel(mi.Point3f, dr.grad(params[key])))
        display(dr.grad(opt['a']))
    else:
        grads.append(dr.grad(opt['a']))
        opt.step()
        angles.append(opt['a'])
        losses.append(l)

#np.save("/home/daniel/loss_n.npy", angles)

In [ ]:
if iters > 1:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 4))
    ax1.set_title("grads")
    ax1.plot(np.array(grads)[:, 0])

    ax2.set_title("angles")
    ax2.plot(np.array(angles)[:, 0])

    ax3.set_title("losses")
    ax3.plot(np.array(losses)[:, 0])
    fig.show()